Допустим, мы хотим узнать, какие книги чаще становятся популярными — художественные или нехудожественные.
Мы можем сформулировать следующую нулевую гипотезу: «Вероятность встретить художественную и нехудожественную
книгу в нашей подборке равна».

In [9]:
import pandas as pd
from scipy.stats import binom_test
df = pd.read_csv('bestsellers with categories.csv')
bins = df['Genre'].value_counts().tolist()
print(bins)
binom_test(bins)

[310, 240]


0.0032232676763761965

Мы проверяем вероятность получить результат статистического теста такой же, как тот, что мы сейчас наблюдаем,
в реальности, где нулевая гипотеза верна. Вероятность небольшая, поскольку она меньше 0.05, — поэтому мы можем отвергнуть
нулевую гипотезу «Вероятность встретить художественную и нехудожественную книгу в нашей подборке равна».

In [7]:
df['Genre'].unique()

array(['Non Fiction', 'Fiction'], dtype=object)

Проверка равенства средних значений в двух выборках (t-тест)

Попробуем узнать, как отличается рейтинг между художественными и нехудожественными текстами.
Есть ли между ними значимые различия?

Для этого мы проведём t-тест. Наша нулевая гипотеза: между средними оценками художественных и нехудожественных книг
нет значимого отличия.

У нас есть данные обо всех бестселлерах. Давайте сформируем из них две случайные выборки, средние в которых сравним.

Сначала нам нужно проверить нормальность распределения средних в выборке. Для этого мы можем использовать тест Шапиро — Уилка.
Однако нам нужно много выборок, чтобы убедиться в том, что среднее значение в них действительно распределено нормально.
Давайте выделим из наших данных 500 выборок и посчитаем для каждой среднее значение.

Для этого мы напишем цикл while, который запустим 500 раз. Для этого будем считать с помощью переменной n количество итераций
цикла. Когда она станет равна 500 — остановимся.

Внутри цикла мы каждый раз будем случайным образом брать сто случайных нехудожественных книг и вычислять их среднюю оценку.
Получившийся результат запишем в список.

In [13]:
from scipy.stats import shapiro
means_fic = []
n = 1

while n < 500:
    n += 1
    sample = df[df['Genre'] == "Non Fiction"].sample(100)['User Rating'].mean()
    means_fic.append(sample)

shapiro(means_fic)

ShapiroResult(statistic=0.9984325170516968, pvalue=0.938539981842041)

Далее нам нужно узнать, равны ли дисперсии по группам. И в зависимости от этого уточнить дополнительный параметр для t-теста.
В Python для проверки критерия Левене мы будем использовать функцию levene из пакета scipy.
Прежде чем проводить тест, снова выделим две случайные выборки, которые будем сравнивать между собой

In [24]:
from scipy.stats import levene
user_rating_fic = df[df['Genre'] == "Fiction"].sample(100)['User Rating']
user_rating_non = df[df['Genre'] == "Non Fiction"].sample(100)['User Rating']
levene(user_rating_fic, user_rating_non)

LeveneResult(statistic=2.90915750915751, pvalue=0.08964603884994217)

Вернёмся к интерпретации результата. Мы видим, что p-value большой, а значит, принимаем нулевую гипотезу о равенстве дисперсий.
Нам нужно отразить это в t-тесте.

In [16]:
from scipy.stats import ttest_ind
ttest_ind(user_rating_fic, user_rating_non, equal_var=True)

Ttest_indResult(statistic=2.424419238829392, pvalue=0.016230942748072505)

Учитываем, что дисперсии не равны, с помощью параметра equal_var, который применяет поправку Уэлча.
Так как p-value меньше 0.05, то мы не можем принять нулевую гипотезу о том, что средние оценки книг не различаются
в зависимости от жанра. То есть жанр связан с оценкой книг.

In [17]:
from statistics import mean
print(mean(user_rating_fic), mean(user_rating_non))

4.648 4.567


Мы видим, что средняя оценка художественных книг оказывается значимо выше, чем нехудожественных.
Это может быть связано с тем, что оценка художественных книг зависит от того, нравятся они 
или не нравятся, а нехудожественных — от того, воспринимаются ли они как полезные или нет.

ANOVA
Теперь мы можем проверить, отличаются ли средние оценки книг за три последних года? 
Поскольку групп наблюдения у нас больше двух, применим тест ANOVA. Наша нулевая гипотеза звучит так:
«За последние три года книги в среднем оценивали одинаково».

In [18]:
from scipy.stats import f_oneway
ur_2019 = df[(df['Year'] == 2019)]['User Rating']
ur_2018 = df[(df['Year'] == 2018)]['User Rating']
ur_2017 = df[(df['Year'] == 2017)]['User Rating']
f_oneway(ur_2019, ur_2018, ur_2017)

F_onewayResult(statistic=2.99211541687636, pvalue=0.05324686656835848)

Опираясь на порог p-value (0.053 > 0.05), мы не можем отбросить нулевую гипотезу. Значит, оценки в топе за год между собой не различаются.

В этом параграфе мы провели три базовых статистических теста с помощью Python.

Вы могли заметить общий принцип: мы формулируем гипотезы и подготавливаем данные, проводим сам тест и интерпретируем результаты.

Тесты, которые вы узнали, можно использовать и для других задач:

вычислить, испортится ли погода на курорте, куда вы собираетесь в отпуск, проанализировав данные за предыдущие года
(анализ временных рядов);
разбить квартиры на рынке недвижимости на группы, изучив, как меняется стоимость в зависимости от разных параметров 
(кластеризация);
узнать, что ещё вам нужно изучить, чтобы зарабатывать больше, — протестировав критерии, которые особенно сильно влияют
на уровень зарплат в вашей индустрии (линейная регрессия).